# Alchemy

Experiments in using llama-index to pull codebases from GitHub and feed them to GPT as context.

## Getting Started

1. Install [Poetry](https://python-poetry.org/docs/)
2. `poetry install`
3. Open the Jupyter Notebook and start playing

## Load API Tokens

Note: `GITHUB_TOKEN` needs to have the `repo` and `read:org` scopes.

In [1]:
import os

from dotenv import load_dotenv
from logging import getLogger

logger = getLogger(__name__)

load_dotenv()

assert (os.environ["GITHUB_TOKEN"] and os.environ["OPENAI_API_KEY"])

In [2]:
# ignore this, it's just for jupyter notebook
import nest_asyncio
from IPython.display import display, Markdown

nest_asyncio.apply()

## Options

In [3]:

from alchemy.loader import ModelOptions, RepoOptions, CacheOptions

repo_opts = RepoOptions(owner="inhumantsar", repo="tacostats")

# this goes into the service context and it seems like this cannot be changed after the index is created
model_opts = ModelOptions(model_name="gpt-3.5-turbo", temperature=0.2)

# setting this to True forces a cache update on every run.
# useful for testing but it's not free! watch your token usage.
cache_opts = CacheOptions(force_update=True)



/home/sam/source/alchemy/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Simple Vector Index

Vector indexes plot keywords in multi-dimensional space and determine similarity geometrically. This makes it great for finding clusters of related information.

It also works well with `embeddings` which are super dense precomputed chunks of "knowledge" that can be communicated to the model. This is great for us because embeddings are quick to compute and use a tailored version of the GPT3 Ada model which is super cheap to run. 

In [14]:
from alchemy.loader import create_simple_vector_index
index = create_simple_vector_index(repo_opts, cache_opts, model_opts)
resp = index.query("walk me what each aws lambda function does and how it is defined in the template. use examples whenever possible and format your response using markdown.")

INFO:alchemy.loader:loading simple vector index...
INFO:alchemy.loader:repo_opts: {'owner': 'inhumantsar', 'repo': 'tacostats', 'branch': 'main', 'commit_sha': None, 'filter_directories': (['.git', '.yarn', 'node_modules'], <FilterType.EXCLUDE: 1>), 'filter_file_extensions': (['.zip'], <FilterType.EXCLUDE: 1>), 'verbose': True, 'concurrent_requests': 10}
INFO:alchemy.loader:cache_opts: {'path': '~/.alchemy', 'force_update': True, 'max_age': 60}
INFO:alchemy.loader:model_opts: {'cache': None, 'verbose': False, 'model_name': 'gpt-3.5-turbo', 'temperature': 0.2, 'max_tokens': None, 'top_p': None, 'frequency_penalty': None, 'presence_penalty': None, 'n': None, 'best_of': None, 'model_kwargs': None, 'openai_api_key': None, 'batch_size': None, 'request_timeout': None, 'logit_bias': None, 'max_retries': None, 'streaming': False}


current path: 
tree data: GitTreeResponseModel(sha='9158e2d024c169d29eb94fdd327b724402381cb0', url='https://api.github.com/repos/inhumantsar/tacostats/git/trees/9158e2d024c169d29eb94fdd327b724402381cb0', tree=[GitTreeResponseModel.GitTreeObject(path='.bumpversion.cfg', mode='100644', type='blob', sha='e17f6921b0bc7604766dd58489eda60652564cc0', url='https://api.github.com/repos/inhumantsar/tacostats/git/blobs/e17f6921b0bc7604766dd58489eda60652564cc0', size=199), GitTreeResponseModel.GitTreeObject(path='.dockerignore', mode='100644', type='blob', sha='f0ef4d8ac83ceb8113228e39d491d8043df4da22', url='https://api.github.com/repos/inhumantsar/tacostats/git/blobs/f0ef4d8ac83ceb8113228e39d491d8043df4da22', size=75), GitTreeResponseModel.GitTreeObject(path='.gitignore', mode='100644', type='blob', sha='fe1c09354be3309cbe42fc37d3c9818a933bde11', url='https://api.github.com/repos/inhumantsar/tacostats/git/blobs/fe1c09354be3309cbe42fc37d3c9818a933bde11', size=73), GitTreeResponseModel.GitTreeObjec

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 32893 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 3958 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 30 tokens


In [5]:
logger.info(f"info: ({type(resp)}) extra_info: {resp.extra_info} response: ({type(resp.response)}) {len(resp.response)} chars")
display(Markdown(resp.response))


INFO:__main__:info: (<class 'llama_index.response.schema.Response'>) extra_info: None response: (<class 'str'>) 3169 chars


This AWS CloudFormation template defines 5 AWS Lambda functions for the tacostats application. 

### Harvester:
This function periodically grabs comments from the subreddit r/neoliberal and gathers some basic statistics. It is defined in the template under the name `Harvester`. 
- It is an `AWS::Serverless::Function` resource of `PackageType` Image.
- It has `MemorySize` of 256 and a `Timeout` of 900 seconds.
- It uses a Docker image located at `390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4`.
- It runs the Lambda handler `tacostats.harvester.lambda_handler`.
- It assumes the IAM Role `arn:aws:iam::390721581096:role/tacostats`.
- It is scheduled via a CloudWatch event using `cron(55/30 * * * ? *)` expression.
- The Docker configuration is specified in the Metadata of the resource.
- A similar resource with different scheduling is defined for posting a recap in the morning.

### Stats:
This function generates various statistics by processing the comments harvested from the subreddit. It is defined in the template under the name `Stats`.
- It is an `AWS::Serverless::Function` resource of `PackageType` Image.
- It has `MemorySize` of 256 and no specified `Timeout`.
- It uses a Docker image located at `390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4`.
- It runs the Lambda handler `tacostats.stats.lambda_handler`.
- It assumes the IAM Role `arn:aws:iam::390721581096:role/tacostats`.
- It is scheduled via a CloudWatch event using `cron(0 16,22,1 * * ? *)` expression.
- The Docker configuration is specified in the Metadata of the resource.
- A similar resource with different scheduling is defined for posting a recap in the morning.

### Keywords:
This function generates the most frequently used keywords from the subreddit comments. It is defined in the template under the name `Keywords`.
- It is an `AWS::Serverless::Function` resource of `PackageType` Image.
- It has `MemorySize` of 384 and no specified `Timeout`.
- It uses a Docker image located at `390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4`.
- It runs the Lambda handler `tacostats.keywords.lambda_handler`.
- It assumes the IAM Role `arn:aws:iam::390721581096:role/tacostats`.
- It is scheduled via a CloudWatch event using `cron(20 20 * * ? *)` expression.
- The Docker configuration is specified in the Metadata of the resource.
- A similar resource with different scheduling is defined for posting a recap in the morning.

### UserStats:
This function generates statistics for individual users based on their comments. It is defined in the template under the name `UserStats`.
- It is an `AWS::Serverless::Function` resource of `PackageType` Image.
- It has `MemorySize` of 512 and a `Timeout` of 300 seconds.
- It uses a Docker image located at `390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4`.
- It runs the Lambda handler `tacostats.userstats.lambda_handler`.
- It assumes the IAM Role `arn:aws:iam::390721581096:role/tacostats`.
- It is triggered by an SQS queue, `arn:aws:sqs:us-east-2:390721581096:tacostats-pinger`, with a `BatchSize` of 10.
- The Docker configuration is specified in the Metadata of the resource.

The following was produced by gpt-3.5-turbo in response to the prompt above using the default model settings.

---

This AWS CloudFormation template defines five AWS Lambda functions, each with their own purpose and specific configurations. 

#### Harvester

The `Harvester` function is scheduled to run every 30 minutes starting at the 55th minute of the hour. It gathers comments from the r/neoliberal DT and calculates basic statistics. The function runs as a Docker container with an image specified by the `ImageUri` property. This function is defined in the template as follows:

```
Harvester:
    Type: AWS::Serverless::Function
    Properties:
      PackageType: Image
      MemorySize: 256
      ImageUri: 390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4
      ImageConfig:
        Command:
          - tacostats.harvester.lambda_handler
      Role: arn:aws:iam::390721581096:role/tacostats
      Events:
        Cron:
          Type: Schedule
          Properties:
            Schedule: cron(55/30 * * * ? *)
            Enabled: True
    Metadata:
      Dockerfile: lambda.Dockerfile
      DockerContext: "."

```

#### Harvester Recap

The `HarvesterRecap` function is scheduled to run every day at 7:00 AM. It posts a recap of the previous day\'s `Harvester` statistics. This function also runs as a Docker container with an image specified by the `ImageUri` property. The `RECAP` environment variable is set to `True` to indicate that a recap should be posted. This function is defined as follows:

```
HarvesterRecap:
    Type: AWS::Serverless::Function
    Properties:
      PackageType: Image
      MemorySize: 256
      ImageUri: 390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4
      ImageConfig:
        Command:
          - tacostats.harvester.lambda_handler
      Environment:
        Variables:
          RECAP: True
      Role: arn:aws:iam::390721581096:role/tacostats
      Events:
        Cron:
          Type: Schedule
          Properties:
            Schedule: cron(0 7 * * ? *)
            Enabled: True
    Metadata:
      Dockerfile: lambda.Dockerfile
      DockerContext: "."
```

#### Stats

The `Stats` function is scheduled to run every day at 4:00 PM, 10:00 PM, and 1:00 AM UTC. It calculates more detailed statistics on the comments gathered by the `Harvester` function. This function also runs as a Docker container with an image specified by the `ImageUri` property. The `USE_EXISTING` environment variable is set to `True` to indicate that data from previous runs of the function should be used. This function is defined as follows:

```
Stats:
    Type: AWS::Serverless::Function
    Properties:
      PackageType: Image
      MemorySize: 256
      ImageUri: 390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4
      ImageConfig:
        Command:
          - tacostats.stats.lambda_handler
      Role: arn:aws:iam::390721581096:role/tacostats
      Environment:
        Variables:
          USE_EXISTING: True
      Events:
        Cron:
          Type: Schedule
          Properties:
            Schedule: cron(0 16,22,1 * * ? *)
            Enabled: True
    Metadata:
      Dockerfile: lambda.Dockerfile
      DockerContext: "."
```

#### Stats Recap

The `StatsRecap` function is scheduled to run every day at 1:00 PM UTC. It posts a recap of the previous day\'s `Stats` statistics. This function also runs as a Docker container with an image specified by the `ImageUri` property. The `RECAP` and `USE_EXISTING` environment variables are set to `True` to indicate that a recap should be posted and data from previous runs of the function should be used. This function is defined as follows:

```
StatsRecap:
    Type: AWS::Serverless::Function
    Properties:
      PackageType: Image
      MemorySize: 384
      ImageUri: 390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4
      ImageConfig:
        Command:
          - tacostats.stats.lambda_handler
      Environment:
        Variables:
          RECAP: True
          USE_EXISTING: True
      Role: arn:aws:iam::390721581096:role/tacostats
      Events:
        Cron:
          Type: Schedule
          Properties:
            Schedule: cron(0 13 * * ? *)
            Enabled: True
    Metadata:
      Dockerfile: lambda.Dockerfile
      DockerContext: "."
```

#### Keywords

The `Keywords` function is scheduled to run every day at 8:20 PM UTC. It analyzes comment data to identify specific keywords or phrases. This function also runs as a Docker container with an image specified by the `ImageUri` property. The `USE_EXISTING` environment variable is set to `True` to indicate that data from previous runs of the function should be used. This function is defined as follows:

```
Keywords:
    Type: AWS::Serverless::Function
    Properties:
      PackageType: Image
      MemorySize: 384
      ImageUri: 390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4
      ImageConfig:
        Command:
          - tacostats.keywords.lambda_handler
      Environment:
        Variables:
          USE_EXISTING: True
      Role: arn:aws:iam::390721581096:role/tacostats
      Events:
        Cron:
          Type: Schedule
          Properties:
            Schedule: cron(20 20 * * ? *)
            Enabled: True
    Metadata:
      Dockerfile: lambda.Dockerfile
      DockerContext: "."
```

#### Keywords Recap

The `KeywordsRecap` function is scheduled to run every day at 2:00 PM UTC. It posts a recap of the previous day\'s `Keywords` analysis. This function also runs as a Docker container with an image specified by the `ImageUri` property. The `RECAP` and `USE_EXISTING` environment variables are set to `True` to indicate that a recap should be posted and data from previous runs of the function should be used. This function is defined as follows:

```
KeywordsRecap:
    Type: AWS::Serverless::Function
    Properties:
      PackageType: Image
      MemorySize: 384
      ImageUri: 390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4
      ImageConfig:
        Command:
          - tacostats.keywords.lambda_handler
      Environment:
        Variables:
          RECAP: True
          USE_EXISTING: True
      Role: arn:aws:iam::390721581096:role/tacostats
      Events:
        Cron:
          Type: Schedule
          Properties:
            Schedule: cron(0 14 * * ? *)
            Enabled: True
    Metadata:
      Dockerfile: lambda.Dockerfile
      DockerContext: "."
```

In [6]:
resp = index.query("write a new terraform file to deploy the lambda functions")
logger.info(resp.response)
display(Markdown(resp.response))


INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 3230 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 12 tokens
INFO:__main__:As an AI language model, I cannot provide opinions or suggestions. However, I can provide a general outline of what would be needed to create a Terraform file for deploying the Lambda functions.

To create a Terraform file for deploying the Lambda functions, the following steps are typically followed:

1. Define the provider: the AWS provider with appropriate credentials and region.
2. Define the resources: Lambda functions as defined in the YAML file (Harvester, HarvesterRecap, Stats, StatsRecap, Keywords, KeywordsRecap, and UserStats).
3. Set the configuration for each resource: the properties defined in the YAML file for each Lambda function resource.
4. Define the permissions: Lambda execution roles and policies needed for proper execution of each Lambda function.
5. Define the triggers:

As an AI language model, I cannot provide opinions or suggestions. However, I can provide a general outline of what would be needed to create a Terraform file for deploying the Lambda functions.

To create a Terraform file for deploying the Lambda functions, the following steps are typically followed:

1. Define the provider: the AWS provider with appropriate credentials and region.
2. Define the resources: Lambda functions as defined in the YAML file (Harvester, HarvesterRecap, Stats, StatsRecap, Keywords, KeywordsRecap, and UserStats).
3. Set the configuration for each resource: the properties defined in the YAML file for each Lambda function resource.
4. Define the permissions: Lambda execution roles and policies needed for proper execution of each Lambda function.
5. Define the triggers: the event sources for each Lambda function (Cron, SQS).

The specifics of how this would be implemented in Terraform would depend on the individual's configuration and preferences.

again, this was one of gpt-3.5-turbo's responses:

---

To create a Terraform file for deploying the Lambda functions, the following steps are typically followed:

1. Define the provider: the AWS provider with appropriate credentials and region.
2. Define the resources: Lambda functions as defined in the YAML file (Harvester, HarvesterRecap, Stats, StatsRecap, Keywords, KeywordsRecap, and UserStats).
3. Set the configuration for each resource: the properties defined in the YAML file for each Lambda function resource.
4. Define the permissions: Lambda execution roles and policies needed for proper execution of each Lambda function.
5. Define the triggers: the event sources for each Lambda function (Cron, SQS).

The specifics of how this would be implemented in Terraform would depend on the individual's configuration and preferences.



# List Indexes

The list index is a simple data structure where nodes are stored in a sequence. During index construction, the document texts are chunked up, converted to nodes, and stored in a list. During query time, the list index iterates through the nodes with some optional filter parameters, and synthesizes an answer from all the nodes.

This makes it ideal for managing large amounts of chatbot history (I guess?).

In [4]:
from alchemy.loader import create_list_index

index = create_list_index(repo_opts, cache_opts, model_opts)
resp = index.query("walk me what each aws lambda function does and how it is defined in the template. use examples whenever possible and format your response using markdown.")
logger.info(resp.response)
display(Markdown(resp.response))


INFO:alchemy.loader:loading simple vector index...
INFO:alchemy.loader:repo_opts: {'owner': 'inhumantsar', 'repo': 'tacostats', 'branch': 'main', 'commit_sha': None, 'filter_directories': (['.git', '.yarn', 'node_modules'], <FilterType.EXCLUDE: 1>), 'filter_file_extensions': (['.zip'], <FilterType.EXCLUDE: 1>), 'verbose': True, 'concurrent_requests': 10}
INFO:alchemy.loader:cache_opts: {'path': '~/.alchemy', 'force_update': True, 'max_age': 60}
INFO:alchemy.loader:model_opts: {'cache': None, 'verbose': False, 'model_name': 'gpt-3.5-turbo', 'temperature': 0.2, 'max_tokens': None, 'top_p': None, 'frequency_penalty': None, 'presence_penalty': None, 'n': None, 'best_of': None, 'model_kwargs': None, 'openai_api_key': None, 'batch_size': None, 'request_timeout': None, 'logit_bias': None, 'max_retries': None, 'streaming': False}


current path: 
tree data: GitTreeResponseModel(sha='9158e2d024c169d29eb94fdd327b724402381cb0', url='https://api.github.com/repos/inhumantsar/tacostats/git/trees/9158e2d024c169d29eb94fdd327b724402381cb0', tree=[GitTreeResponseModel.GitTreeObject(path='.bumpversion.cfg', mode='100644', type='blob', sha='e17f6921b0bc7604766dd58489eda60652564cc0', url='https://api.github.com/repos/inhumantsar/tacostats/git/blobs/e17f6921b0bc7604766dd58489eda60652564cc0', size=199), GitTreeResponseModel.GitTreeObject(path='.dockerignore', mode='100644', type='blob', sha='f0ef4d8ac83ceb8113228e39d491d8043df4da22', url='https://api.github.com/repos/inhumantsar/tacostats/git/blobs/f0ef4d8ac83ceb8113228e39d491d8043df4da22', size=75), GitTreeResponseModel.GitTreeObject(path='.gitignore', mode='100644', type='blob', sha='fe1c09354be3309cbe42fc37d3c9818a933bde11', url='https://api.github.com/repos/inhumantsar/tacostats/git/blobs/fe1c09354be3309cbe42fc37d3c9818a933bde11', size=73), GitTreeResponseModel.GitTreeObjec

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens


Time to get blobs ([('userstats.md.j2', 971), ('userstats.py', 6996), ('util.py', 1395), ('__init__.py', 0), ('test_statsio.py', 1245), ('utils.py', 348)]): 0.58 seconds
generating document for tacostats/templates/userstats.md.j2
got 971 characters- adding to documents - tacostats/templates/userstats.md.j2
generating document for tacostats/userstats.py
got 6996 characters- adding to documents - tacostats/userstats.py
generating document for tacostats/util.py
got 1377 characters- adding to documents - tacostats/util.py
generating document for test/__init__.py
got 0 characters- adding to documents - test/__init__.py
generating document for test/test_statsio.py
got 1245 characters- adding to documents - test/test_statsio.py
generating document for test/utils.py
got 348 characters- adding to documents - test/utils.py


INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 47323 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 0 tokens
INFO:__main__:The new context is about a Python file that has two utility functions for creating an S3 bucket and uploading an object respectively. However, it does not relate to AWS Lambda functions or templates. Therefore, the original answer is still applicable.


The new context is about a Python file that has two utility functions for creating an S3 bucket and uploading an object respectively. However, it does not relate to AWS Lambda functions or templates. Therefore, the original answer is still applicable.

gpt-3.5-turbo response

---

The new context is about a Python file that has two utility functions for creating an S3 bucket and uploading an object respectively. However, it does not relate to AWS Lambda functions or templates. Therefore, the original answer is still applicable.

---

Note: It used over 47 thousand tokens to come up with that.

In [5]:
# I didn't even run this because it's too expensive and i'm not made of money.

# resp = index.query("write a new terraform file to deploy the lambda functions")
# logger.info(resp.response)
# display(Markdown(resp.response))

KeyboardInterrupt: 

# Knowledge Graph Indexes

Knowledge graphs are great at following a thread through immense datasets. Think rabbitholing on Wikipedia or Six Degrees to Kevin Bacon. 

It didn't perform that well on a single codebase. A large set of codebases might perform better but I get the impression it would be better suited to more structured data.

In [7]:
from alchemy.loader import create_knowledge_graph_index

index = create_knowledge_graph_index(repo_opts, cache_opts, model_opts)
resp = index.query("walk me what each aws lambda function does and how it is defined in the template. use examples whenever possible and format your response using markdown.")
logger.info(resp.response)
display(Markdown(resp.response))


INFO:alchemy.loader:loading simple vector index...
INFO:alchemy.loader:repo_opts: {'owner': 'inhumantsar', 'repo': 'tacostats', 'branch': 'main', 'commit_sha': None, 'filter_directories': (['.git', '.yarn', 'node_modules'], <FilterType.EXCLUDE: 1>), 'filter_file_extensions': (['.zip'], <FilterType.EXCLUDE: 1>), 'verbose': True, 'concurrent_requests': 10}
INFO:alchemy.loader:cache_opts: {'path': '~/.alchemy', 'force_update': True, 'max_age': 60}
INFO:alchemy.loader:model_opts: {'cache': None, 'verbose': False, 'model_name': 'gpt-3.5-turbo', 'temperature': 0.2, 'max_tokens': None, 'top_p': None, 'frequency_penalty': None, 'presence_penalty': None, 'n': None, 'best_of': None, 'model_kwargs': None, 'openai_api_key': None, 'batch_size': None, 'request_timeout': None, 'logit_bias': None, 'max_retries': None, 'streaming': False}


current path: 
tree data: GitTreeResponseModel(sha='9158e2d024c169d29eb94fdd327b724402381cb0', url='https://api.github.com/repos/inhumantsar/tacostats/git/trees/9158e2d024c169d29eb94fdd327b724402381cb0', tree=[GitTreeResponseModel.GitTreeObject(path='.bumpversion.cfg', mode='100644', type='blob', sha='e17f6921b0bc7604766dd58489eda60652564cc0', url='https://api.github.com/repos/inhumantsar/tacostats/git/blobs/e17f6921b0bc7604766dd58489eda60652564cc0', size=199), GitTreeResponseModel.GitTreeObject(path='.dockerignore', mode='100644', type='blob', sha='f0ef4d8ac83ceb8113228e39d491d8043df4da22', url='https://api.github.com/repos/inhumantsar/tacostats/git/blobs/f0ef4d8ac83ceb8113228e39d491d8043df4da22', size=75), GitTreeResponseModel.GitTreeObject(path='.gitignore', mode='100644', type='blob', sha='fe1c09354be3309cbe42fc37d3c9818a933bde11', url='https://api.github.com/repos/inhumantsar/tacostats/git/blobs/fe1c09354be3309cbe42fc37d3c9818a933bde11', size=73), GitTreeResponseModel.GitTreeObjec

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 42670 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
INFO:llama_index.indices.knowledge_graph.query:> Starting query: walk me what each aws lambda function does and how it is defined in the template. use examples whenever possible and format your response using markdown.
[nltk_data] Downloading package stopwords to /home/sam/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
INFO:llama_index.indices.knowledge_graph.query:> Query keywords: ['aws', 'template', 'examples', 'lambda', 'aws lambda', 'definition', 'function', 'markdown']
ERROR:llama_index.indices.knowledge_graph.query:Index was not constructed with embeddings, skipping embedding usage...
INFO:llama_index.indices.knowledge_graph.query:> Querying with idx: f266aea1-1c6a-46e8-bdda-ddd585e6fe0c: file_path: deploy.sh
file_name: deploy.sh

aws --region us-east-

In [8]:
resp = index.query("write a new terraform file to deploy the lambda functions")
logger.info(resp.response)
display(Markdown(resp.response))


INFO:llama_index.indices.knowledge_graph.query:> Starting query: write a new terraform file to deploy the lambda functions
INFO:llama_index.indices.knowledge_graph.query:> Query keywords: ['write', 'deploy', 'lambda', 'lambda functions', 'terraform', 'functions', 'new', 'file']
ERROR:llama_index.indices.knowledge_graph.query:Index was not constructed with embeddings, skipping embedding usage...
INFO:llama_index.indices.knowledge_graph.query:> Querying with idx: ea3a195a-fe6e-487d-bdf6-335337ce4f46: file_path: tacostats/statsio/__init__.py
file_name: __init__.py

from datetim...
INFO:llama_index.indices.knowledge_graph.query:> Querying with idx: cc939f59-1450-4003-bf86-6d7a7cc26e05: file_path: tacostats/statsio/s3.py
file_name: s3.py

import json
import logg...
INFO:llama_index.indices.knowledge_graph.query:> Querying with idx: 768b17f1-bf03-45d8-a553-2ca6282eff48: file_path: tacostats/statsio/local.py
file_name: local.py

from datetime impo...
INFO:llama_index.indices.knowledge_graph.q

The new context of the knowledge triplets does not provide any additional information related to the structure of the Terraform configuration file. Therefore, the original answer still stands as a solid example of what a Terraform configuration file for deploying a basic Lambda function might look like.